In [35]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [36]:
!chmod 600 /root/.kaggle/kaggle.json

In [37]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
utkface-new.zip: Skipping, found more recently modified local copy (use --force to force download)


In [38]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [39]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [40]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [41]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [42]:
len(age)

23708

In [43]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [44]:
df.shape

(23708, 3)

In [45]:
df.head()

,age,gender,img
0,26,0,26_0_1_20170116153040656.jpg.chip.jpg
1,23,1,23_1_2_20170116172848833.jpg.chip.jpg
2,50,0,50_0_0_20170117190657761.jpg.chip.jpg
3,9,0,9_0_0_20170110220413289.jpg.chip.jpg
4,40,1,40_1_1_20170116223441453.jpg.chip.jpg


In [46]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [47]:
train_df.shape

(20000, 3)

In [48]:
test_df.shape

(3708, 3)

In [49]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [61]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=folder_path,
    x_col="img",
    y_col=["age", "gender"],
    target_size=(200, 200),
    batch_size=32,
    class_mode="raw"
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=folder_path,
    x_col="img",
    y_col=["age", "gender"],
    target_size=(200, 200),
    batch_size=32,
    class_mode="raw"
)


Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [62]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [63]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [64]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [65]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [66]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [67]:
def to_dict_generator(gen):
    for x, y in gen:
        yield x, {
            "age": y[:, 0],
            "gender": y[:, 1]
        }
train_gen_fixed = to_dict_generator(train_generator)
test_gen_fixed  = to_dict_generator(test_generator)


In [68]:
model.fit(
    train_gen_fixed,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_data=test_gen_fixed,
    validation_steps=len(test_generator)
)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 253s 386ms/step - age_loss: 15.8184 - age_mae: 15.8184 - gender_accuracy: 0.5086 - gender_loss: 1.3827 - loss: 152.7029 - val_age_loss: 14.8493 - val_age_mae: 14.8464 - val_gender_accuracy: 0.5175 - val_gender_loss: 0.6927 - val_loss: 83.4209
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 230s 369ms/step - age_loss: 15.0125 - age_mae: 15.0125 - gender_accuracy: 0.5242 - gender_loss: 0.6971 - loss: 84.0259 - val_age_loss: 15.2162 - val_age_mae: 15.2162 - val_gender_accuracy: 0.5145 - val_gender_loss: 0.6928 - val_loss: 83.8016
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 230s 369ms/step - age_loss: 14.7926 - age_mae: 14.7926 - gender_accuracy: 0.5218 - gender_loss: 0.6960 - loss: 83.7009 - val_age_loss: 15.2902 - val_age_mae: 15.2902 - val_gender_accuracy: 0.5154 - val_gender_loss: 0.6929 - val_loss: 83.8873
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 238s 382ms/step - age_loss: 14.9817 - age_mae: 14.9817 - gender_accuracy: 0.5247 - gender_loss: 0.6939 - loss: 83